In [1]:
#! pip install py2neo;
! pip install neo4j
from py2neo import Graph
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import seaborn as sns
import numpy as np

    100% |████████████████████████████████| 71kB 33.1MB/s ta 0:00:01
    100% |████████████████████████████████| 81kB 40.6MB/s ta 0:00:01
    100% |████████████████████████████████| 184kB 30.8MB/s ta 0:00:01
  Using cached https://files.pythonhosted.org/packages/87/61/2dfea88583d5454e3a64f9308a686071d58d59a55db638268a6413e1eb6d/prompt_toolkit-2.0.10-py3-none-any.whl
    100% |████████████████████████████████| 849kB 32.4MB/s ta 0:00:01
  Running setup.py bdist_wheel for py2neo ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/86/7d/e3/35bd456d80e796ba1f98730b629352ebab5294c942e8fdd826
  Running setup.py bdist_wheel for neobolt ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/42/54/00/270de525f8f966d864bd3a0224d38dca2573b53c1a54267720
  Running setup.py bdist_wheel for neotime ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/d4/b4/c4/ab867cd0ac9e667f8cd269f9364fd1cb8ee75fe50f5ce97c79
Successfully built py2neo neobolt neotime
ipython 6.5.0 ha

In [2]:
#graph = Graph("bolt://3.220.233.169:7687", auth=("neo4j", "i-0e23d19f0d8795714"))

In [ ]:
uri = "bolt://3.220.233.169:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "i-0e23d19f0d8795714"))

In [61]:
user_id=195103
biz_id=17

In [64]:
friend_count=graph.run("""MATCH (c:User)
RETURN ID(c) as ident, apoc.node.degree(c, 'FRIEND') as count
""").to_data_frame()

In [65]:
friend_count.describe()

,count,ident
count,1.637138e+06,1.637138e+06
mean,2.754502e-01,3.490227e+06
std,1.131968e+00,2.408194e+06
min,0.000000e+00,1.950860e+05
25%,0.000000e+00,1.344378e+06
50%,0.000000e+00,3.063662e+06
75%,0.000000e+00,5.402947e+06
max,3.450000e+02,8.512231e+06


In [68]:
friend_count['count'].quantile(.80)

1.0

In [66]:
friend_count.loc[friend_count['count']==0].shape

(1312387, 2)

In [38]:
friends=graph.run("""
CALL algo.unionFind.stream('User', 'FRIEND')
YIELD nodeId, setId
""").to_data_frame()

In [52]:
clusters=friends.loc[friends.setId.duplicated(keep=False)]

In [59]:
clusters.setId.value_counts().describe()

count    99316.000000
mean         3.269876
std          5.861368
min          2.000000
25%          2.000000
50%          2.000000
75%          3.000000
max        426.000000
Name: setId, dtype: float64